In [1]:
from __future__ import print_function
import os
import sys
import numpy as np
import scipy.stats as sps
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## NULL HYPOTHESIS: the % of former prisoners employed after release is the same or lower for candidates who participated in the program as for the control group, significance level $p=0.05$

$H_0: P_0 - P_1 \geq 0$

$H_a: P_0 - P_1 < 0$

$\alpha = 0.05$

this is a TEST OF PROPORTIONS. we use the Binomial distribution since it is a yes/no (bernulli) test for each subject: the former inmate was or was not ever employed in a CEO transitional job (second row in the table above):

$P_0=0.035, P_1=0.701$

In [2]:
alpha = 0.05
# we like fractions better then percentages. as a rule of thumb, either use fractions or counts
P_0 = 3.5 * 0.01 
P_1 = 70.1 * 0.01

if P_0 - P_1 >= 0:
    # we are done
    print ("the Null holds")
else:
    print ("we must assess the statistical significance")

n_0 = 409
n_1 = 564

#lets get the counts by multiplying by the sample size
Nt_0 = P_0 * n_0
Nt_1 = P_1 * n_1

we must assess the statistical significance


## 2 samples, categorical data

## TWO OPTIONS z test, or chi-square test.

## START WITH Z TEST

## the z test compares the stanrard deviation of the expected distribution and the observed result. it tells you literally how many standard deviations from the tail an observation is, under the _assumption of normality

must define the sample standard deviation

In [3]:
#define the sample proportion first
sp = (P_0 * n_0 + P_1 * n_1) / (n_1 + n_0)
print (sp)

0.421047276465


## standard deviation of the sampling distribution the distribution is Binomial, the binomail stdev is

(see a proof here!: http://stats.stackexchange.com/questions/29641/standard-error-for-the-mean-of-a-sample-of-binomial-random-variables!):

$\sqrt{\frac{p(1 - p)}{n}}$

for 2 samples this becomes

$\sqrt{ \frac{ \hat{p}(1 - \hat{p})} {n0} + \frac{ \hat{p}(1 - \hat{p})} {n1} }$

cfr: page 138 of Statistics in a Nutshell, eq. 5.12 and here http://stattrek.com/hypothesis-test/difference-in-proportions.aspx?Tutorial=AP

$\hat{p} =\frac{p_0  n_0 + p_1  n_1}{n_0+n_1}$

In [4]:
# I am goonna create a little one line function to calculate the standard error
# and to calculate p

phat = lambda p0, p1, n0, n1: (p0 * n0 + p1 * n1) / (n0 + n1)
#standard error
serr = lambda p, n0, n1: np.sqrt(p * (1 - p) * (1.0 / n0 + 1.0 / n1)) 

## z score: how many standard deviation away from the population parameter is my statistic?

$z=\frac{P_1-P_0}{\sigma}$

In [5]:
zscore = lambda p0, p1, s : (p0 - p1) / s
z_2y = zscore(P_1, P_0, serr(phat(P_0, P_1, n_0, n_1), n_0, n_1))
print (z_2y)

20.7697865408


note that using p0-p1 or p1-p0 at the numerator is equivalent because the standardizes normal value of z has mean 0 (see image below) so that we can use the absolute value of the z score, or equivalently look for $P[Z<z]$ if z is positive, and $P[Z>z]$ if z is negative.

## if $p <\alpha$ : reject H0

## IMPORTANT!! note that this P in the bottom line of the table is not the p-value, but¶

## p-value = 1-P

In [6]:
## p-value for employment after 2 years: 
## since the largest number we read off the table for is (way) smaller 
## than the value for our statistic 
## our p-value will be smaller than it would be if calculated using 
## (e.g.) .9998 (and in fact using 1.0000 which is the largest number 
## in the table). Using 0.9998 is a **conservative** approach. 

p_2y = 1 - sps.norm.cdf(z_2y)

def report_result(p,a):
    print ('is the p value ' + 
           '{0:.2f} smaller than the critical value {1:.2f}?'.format(p,a))
    if p < a:
        print ("YES!")
    else: 
        print ("NO!")
    
    print ('the Null hypothesis is {}'.format(\
                            'rejected' if p < a  else 'not rejected') )

    
report_result(p_2y, alpha)

is the p value 0.00 smaller than the critical value 0.05?
YES!
the Null hypothesis is rejected


what if we used the values for where the former inmate was or was not "Convicted of a felony" (row 10) in the Recidivism (Years 1-3)?

## Null hypothesis

The percentage of participants in the Center for Employment Opportunities convicted of a felony after 3 years is at least as large as the percentage of that for non-participants.

$H_0: FC_1 \geq FC_0$

$H_a: FC_1 < FC_0$

$P_0 = 0.10, P_1= 0.117$

look up data table and insert the appropriate values to get the appropriate result! you can use the functions I defined above, with different arguments.

Now lets do it with the $\chi^2$ test
this analysis can also be done with the $\chi^2$ test, and the  $\chi^2$ distribution,

The chisq statistics tests the statistics calculated as :

$\chi^2 = \sum_{i} \frac{(observation_i - expectation_i)^2}{expectation_i}$

against a chi sq distribution.

If we talk about sample fractions that is

$\chi^2 = \sum_i \frac{(f_{i,observed} - f_{i,expectated})^2}{f_{i,expected}}$

Where i indicates the sum over each cell.

turns out this quantity is distributed according to a chi square distribution, so if i get the 

$\chi^2$ statistics i can compare it to the full chisq distribution and see how far in the tail it is

let's see what the chi sq statistics says about the conviction for felonies (row 10)

<table><tr><td>convicted of a felony</td><td>yes</td><td>no</td><td></td></tr>
<tr><td>test sample</td><td>0.1 $\times$ 568</td><td>0.9 $\times$ 568</td><td>586</td></tr>
<tr><td>control sample</td><td>0.117 $\times$ 409</td><td>0.883 $\times$ 409</td><td>409</td></tr>
<tr><td>total</td><td>104.653</td><td>872.347</td><td>977</td></tr></table>

In [7]:
FC_1 = 0.1
FC_0 = 0.117
rs_1 = 568
rs_0 = 409
FChat = phat(FC_0, FC_1, rs_0, rs_1)

def chisq_sum(ex_avg, sample):
    sum_devs = 0
    for i in range(0, len(sample)):
        expect = sample[i][1] * ex_avg
        obs = sample[i][0] * sample[i][1]
        sum_devs += (expect - obs) ** 2 / expect
    return sum_devs

fel_result = chisq_sum(FChat, np.array([[FC_1, rs_1], [FC_0, rs_0]]))
print(fel_result)

0.641531231785


In [8]:
print(1 - sps.chi2.cdf(fel_result, 1))

0.423156859321


$0.42>0.05$, therefore, the null hypothesis is not rejected by the $\chi^2$ method.

Now to try a $Z$-test.

In [9]:
err_FC = serr(FChat, rs_0, rs_1)
Z_FC = zscore(FC_0, FC_1, err_FC)
p_FC = 1 - sps.norm.cdf(Z_FC)
report_result(p_FC, alpha)

is the p value 0.20 smaller than the critical value 0.05?
NO!
the Null hypothesis is not rejected


Now to apply $\chi^2$ to the employment cell.

<table><tr><td>ever employed</td><td>yes</td><td>no</td><td></td></tr>
<tr><td>test sample</td><td>0.701 $\times$ 564</td><td>0.299 $\times$ 564</td><td>584</td></tr>
<tr><td>control sample</td><td>0.035 $\times$ 409</td><td>0.965 $\times$ 409</td><td>409</td></tr>
<tr><td>total</td><td>406.679</td><td>563.321</td><td>973</td></tr></table>

In [10]:
emp_result = chisq_sum(phat(P_0, P_1, n_0, n_1), np.array([[P_0, n_0], [P_1, n_1]]))
emp_result

249.75096076684426

In [13]:
emp_chi2_pval = 1 - sps.chi2.cdf(emp_result, 1)
print(emp_chi2_pval)

0.0


For the effect on employment, the $\chi^2$ test also has a $p$-value less than threshold.